# 프로세스 정리

로그인 -> 구독해 있는 채널 확인 + 원하는 채널만 분류
    -> 원하는 범위의 채널 메시지 불러오기 + 메시지 분류 및 json 저장

In [10]:
from telethon.tl.types import InputPeerEmpty, Channel, User, Chat
from telethon.tl.functions.channels import GetFullChannelRequest
from telethon.tl.functions.messages import GetDialogsRequest
from datetime import datetime, timedelta, timezone
from telethon import TelegramClient, events
from dotenv import load_dotenv
from pathlib import Path
import asyncio
import json
import time
import os

load_dotenv()

# Telegram API 정보
api_id = os.getenv('TELEGRAM_API_ID')
api_hash = os.getenv('TELEGRAM_API_HASH')
phone = os.getenv('TELEGRAM_PHONE')

client = TelegramClient('telegram_session', api_id, api_hash)

## 로그인

In [11]:
await client.connect()

if not await client.is_user_authorized():
    await client.send_code_request(phone)
    print(f"{phone}으로 코드를 보냈습니다. 코드를 입력하세요:")
    code = input()
    await client.sign_in(phone, code)

## 구독해 있는 채널 확인 + 원하는 채널만 분류 

In [12]:
avoid_channels_name = ['ㅇㅇ','ㄱㄴㄷ','아ㅏ아','코백남 공지방','화장품, 실리콘투, 뷰티']
channels = []

dialogs = await client.get_dialogs()

for dialog in dialogs:
    entity = dialog.entity
    if isinstance(entity, Channel):
        if entity.title not in avoid_channels_name:
            channels.append(entity.id)

print(channels)

[1545256759, 1396820120, 1386345244, 1267620333, 2342739984, 1909745040, 1923637862, 1839490666, 2172249444, 1379897604, 1363666182, 1401647353, 1260023521, 1576521642, 1451111390, 2240178633, 1917610401, 1356118022, 1798170763, 1989467167, 1326994859, 1932462219, 1245319176, 1890360860, 1720065182, 1127010518, 1682527794, 1191697168, 2039438207, 2014051067, 1967687669, 1150586161, 2009765098, 1378197756, 1657785983, 1563539882, 1410262947, 1306052516, 2313632611, 1389529013, 2042637475, 1953325362, 1192351807, 1652098893, 1738234843, 1487913461, 1219549285, 1390138428, 1590774317, 1778838617, 1217109609, 1619402703, 1519612053, 1656364050, 1472429824, 1962529222, 1506140857, 1514595150, 1501645737, 1185561205, 1124650657, 1954342371, 1500265128, 1066938528, 1771079971, 1600798466, 1147595657, 1235226464, 2197517750, 1086116949, 1527329083, 1784919258, 1346864141, 1176773168, 1127984656, 1667546254, 1331790320, 1257359341, 2138524075, 1550772257, 1234537745, 1080519355, 1691482032, 139

## 원하는 범위의 채널 메시지 불러오기 + 메시지 분류 및 json 저장

In [13]:
# 색터에 해당하는 채널 찾기

current_dir = Path.cwd().parent
data_path = current_dir / "data" / "sectors.json"

# 섹터 나누는 기준 불러오기
with open(data_path, "r") as f:
    sectors = json.load(f)

# 해당 메시지에 존재하는 섹터 찾기
async def categorize_message_by_sector(messages):
    found_sectors = []
    for sector, keywords in sectors.items():
        for keyword in keywords:
            if keyword in messages:
                found_sectors.append(sector)
                break

    return list(set(found_sectors))

In [14]:
# 데이터 형식 정의
from collections import defaultdict
from dataclasses import dataclass

@dataclass
class Post:
    time: str
    content: str
    views: int
    replies: int

# 3중 중첩 defaultdict 생성
telegram_crawling = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            lambda: {
                "posts": [],
                "subscribe": 0
            }
        )
    )
)

In [15]:
"""
특정 채널에서 지정된 날짜 범위 내의 모든 메시지를 가져오는 함수

매개변수:
- client: 텔레그램 클라이언트 인스턴스
- channel_id: 텔레그램 채널 ID (정수)
- start_date: 시작 날짜 (datetime 객체)
- end_date: 종료 날짜 (datetime 객체)
- telegram_crawling: 데이터를 담을 칸
"""

async def get_channel_messages(client, channel_id, start_date, end_date, telegram_crawling):
    try:
        # 채널 엔티티 가져오기
        channel = await client.get_entity(channel_id)

        # 전체 채널 정보 가져오기
        try:
            full_channel = await client(GetFullChannelRequest(channel=channel))
            subscribers_count = full_channel.full_chat.participants_count
        except Exception as e:
            print(f"{e}")
        
        # 메시지 가져오기 (end_date부터 역순으로)
        async for message in client.iter_messages(
            channel, 
            offset_date=end_date,
            reverse=False
        ):
            # 시작 날짜보다 이전 메시지면 중단
            if message.date < start_date:
                break
                
            # 종료 날짜보다 이후 메시지면 건너뛰기
            if message.date > end_date:
                continue

            #섹터 확인
            sectors = await categorize_message_by_sector(message.message)

            date_str = message.date.strftime('%Y-%m-%d')
            channel_title = channel.title
            
            # 섹터가 없으면 다음 메시지로
            if not sectors:
                continue
            
            for sector in sectors:
                if sector not in telegram_crawling:
                    telegram_crawling[sector] = {}
                
                if date_str not in telegram_crawling[sector]:
                    telegram_crawling[sector][date_str] = {}
                
                channel_title = channel.title
                if channel_title not in telegram_crawling[sector][date_str]:
                    telegram_crawling[sector][date_str][channel_title] = {
                        "posts": [],
                        "subscribers": subscribers_count
                    }
                
                # 게시글 정보 저장
                post_info = {
                    "time": message.date.strftime('%H:%M:%S'),
                    "content": message.message,
                    "views": message.views if hasattr(message, 'views') else 0,
                    "replies": len(await client.get_messages(channel, reply_to=message.id)) if message.replies else 0
                }
                
                telegram_crawling[sector][date_str][channel_title]["posts"].append(post_info)
            
            await asyncio.sleep(0.5)
        
        
    except Exception as e:
        import traceback
        print(f"채널 ID: {channel_id}, 채널 엔티티: {channel if 'channel' in locals() else '없음'}")
        print(f"메시지 가져오기 중 오류 발생: {e}")
        print(traceback.format_exc())


In [18]:
end_date = datetime.now(timezone(timedelta(hours=9)))
start_date = end_date - timedelta(days=7)
telegram_crawling = {}

print(len(channels))

async def process_all_channels():
    n=1
    for channel in channels:
        try:
            print(f"채널 ID: {n}번 채널 처리 시작")
            await get_channel_messages(client=client, channel_id=channel,
                         start_date=start_date, end_date=end_date, sector_data=telegram_crawling)
            print(f"처리 완료 \n")
        except Exception as e:
            import traceback
            print(f"채널 ID: {n}번: {channel} 처리 중 오류 발생: {e}")
            print(traceback.format_exc())  # 상세 에러 스택 출력
        n += 1

await process_all_channels()

163
채널 ID: 1번 채널 처리 시작
채널 ID: 1545256759, 채널 엔티티: Channel(id=1545256759, title='중앙기획전략부', photo=ChatPhoto(photo_id=6089268903168557894, dc_id=5, has_video=False, stripped_thumb=b'\x01\x08\x08\xa8\xa8\x8d\x03l\x8c\xef\xdc1\xf8\xd1E\x15\x1c\xcd\x17k\x9f'), date=datetime.datetime(2024, 8, 23, 8, 42, 6, tzinfo=datetime.timezone.utc), creator=False, left=False, broadcast=True, verified=False, megagroup=False, restricted=False, signatures=False, min=False, scam=False, has_link=True, has_geo=False, slowmode_enabled=False, call_active=False, call_not_empty=False, fake=False, gigagroup=False, noforwards=False, join_to_send=False, join_request=False, forum=False, stories_hidden=False, stories_hidden_min=False, stories_unavailable=True, signature_profiles=False, access_hash=6197651663426960391, username='getfeed', restriction_reason=[], admin_rights=None, banned_rights=None, default_banned_rights=None, participants_count=None, usernames=[], stories_max_id=None, color=None, profile_color=None, emo

In [ ]:
# JSON 파일로 저장
with open('../data/telegram_crawling.json', 'w', encoding='utf-8') as f:
    json.dump(telegram_crawling, f, ensure_ascii=False, indent=2)

print("telegram_crawling가 JSON 파일로 저장되었습니다.")

sector_data가 JSON 파일로 저장되었습니다.


In [ ]:
await client.disconnect()